# Spark API Exercises
_____

In [1]:
# Creating a Spark session first:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Exercise 1:

Within your `codeup-data-science` directory, create a new repo named `spark-exercises`. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

Create a spark data frame that contains your favorite programming languages.

- Create a dataframe with one column named `language`
> Hint: Start with a pandas dataframe. Maybe use a dictionary?
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [4]:
import pandas as pd
import numpy as np

np.random.seed(456)

lang_df = pd.DataFrame(
    dict(n=['Python', 'HTML', 'JavaScript', 'Java', 'TrumpScript'])
)
lang_df = spark.createDataFrame(lang_df)
lang_df.show()

+-----------+
|          n|
+-----------+
|     Python|
|       HTML|
| JavaScript|
|       Java|
|TrumpScript|
+-----------+



In [7]:
lang_df.printSchema()

root
 |-- n: string (nullable = true)



In [8]:
print('DataFrame Shape: ', lang_df.count(), 'by ', len(lang_df))

TypeError: object of type 'DataFrame' has no len()